<img src="https://s3-sa-east-1.amazonaws.com/preditiva.ai/diversos/preditiva_assinatura.jpg">

# Técnicas de Validação Cruzada

## Demonstração

### Importação das bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#métrica de performance
from sklearn.metrics import roc_auc_score

#modelo a ser treinado
from sklearn.tree import DecisionTreeClassifier

### Importação da base de dados

In [ ]:
df = pd.read_csv('base_rh.csv',sep=";")

In [ ]:
df.head()

### Particionamento usando K-Fold

In [ ]:
# Separação das variáveis explicativas e a variável target
x = df.drop('Funcionário_deixou_a_empresa', axis=1)
y = df['Funcionário_deixou_a_empresa']

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
particoes = KFold(n_splits = 5)

In [ ]:
for train_index, test_index in particoes.split(x.head(10)):
    print(train_index, len(train_index), test_index, len(test_index)) # mostra os índices do DataFrame X com 10 linhas

Perceba que o percentual da base que fica para teste é 1/k. **No exemplo, se k = 5 então o particionamento fica em 80% para Treino e 20% para Teste.**

### Treino de modelos usando K-Fold

In [ ]:
particoes = KFold(n_splits = 5, shuffle = True, random_state = 42) 

In [ ]:
# Separação das variáveis explicativas e a variável target
x = df.drop('Funcionário_deixou_a_empresa', axis=1)
y = df['Funcionário_deixou_a_empresa']

In [ ]:
roc_train = []
roc_test = []

# Criando as variáveis dummies
x = pd.get_dummies(x)

for train_index, test_index in particoes.split(x):
    
    # Separa a base    
    X_train, X_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    #Treina o modelo
    modelo = DecisionTreeClassifier(max_depth=2)
    modelo.fit(X_train, y_train)
    
    # AUC em Treino
    prob_train = modelo.predict_proba(X_train)[:,1]
    roc_train.append(roc_auc_score(y_train, prob_train))
    
    # AUC em Teste
    prob_test = modelo.predict_proba(X_test)[:,1]
    roc_test.append(roc_auc_score(y_test, prob_test))
    
    
resultado = pd.DataFrame({"AUC em Treino":roc_train, "AUC em Teste":roc_test})
resultado

In [ ]:
resultado.describe().loc[['mean','std']]

O valor médio de AUC em Teste ficou em 0,69 com um desvio padrão maior que o AUC de Treino. Perceba que se não usássemos validação cruzada, poderíamos cair na primeira partição e achar que o modelo não generaliza bem (AUC em Teste = 0,58) quando na média ele generaliza (AUC médio em Teste = 0,69).

### Como melhorar a Validação Cruzada?

Um dos problemas da aleatoriedade do particionamento é que não garantimos que Treino e Teste terão a mesma **"Distribuição"**. Veja um exemplo:

In [ ]:
roc_train = []
roc_test = []
target_train = []
target_test = []

# Criando as variáveis dummies
x = pd.get_dummies(x)

for train_index, test_index in particoes.split(x):
    
    # Separa a base    
    X_train, X_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Target em Treino e Teste
    target_train.append(np.where(y_train == 'Sim',1,0).mean())
    target_test.append(np.where(y_test == 'Sim',1,0).mean())
    
    #Treina o modelo
    
    modelo = DecisionTreeClassifier(max_depth=2)
    modelo.fit(X_train, y_train)
    
    # AUC em Treino
    prob_train = modelo.predict_proba(X_train)[:,1]
    roc_train.append(roc_auc_score(y_train, prob_train))
    
    # AUC em Teste
    prob_test = modelo.predict_proba(X_test)[:,1]
    roc_test.append(roc_auc_score(y_test, prob_test))
    
    
resultado = pd.DataFrame({"Target em Treino":target_train,"Target em Teste":target_test,"AUC em Treino":roc_train, "AUC em Teste":roc_test})
resultado

Veja que a **distribuição do Target entre modelos não é o mesmo**... Isso pode afetar a mensuração da métrica de performance. 

### K-Fold com Estratificação

Um forma de minimizar esse problema da diferença de Targets entre as partições é "estratificar" as partições, isto é, garantir que as partições de treino e teste tenha aproximadamente a mesma quantidade proporcional de Targets. Veja como fazer:

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
particoes = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42) 

In [ ]:
# Separação das variáveis explicativas e a variável target
x = df.drop('Funcionário_deixou_a_empresa', axis=1)
y = df['Funcionário_deixou_a_empresa']

In [ ]:
roc_train = []
roc_test = []
target_train = []
target_test = []

# Criando as variáveis dummies
x = pd.get_dummies(x)

for train_index, test_index in particoes.split(x,y): # Aqui colocamos o parâmetro y para que o particionador saiba calcular as proporções
    
    # Separa a base    
    X_train, X_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Target em Treino e Teste
    target_train.append(np.where(y_train == 'Sim',1,0).mean())
    target_test.append(np.where(y_test == 'Sim',1,0).mean())
    
    #Treina o modelo
    
    modelo = DecisionTreeClassifier(max_depth=2)
    modelo.fit(X_train, y_train)
    
    # AUC em Treino
    prob_train = modelo.predict_proba(X_train)[:,1]
    roc_train.append(roc_auc_score(y_train, prob_train))
    
    # AUC em Teste
    prob_test = modelo.predict_proba(X_test)[:,1]
    roc_test.append(roc_auc_score(y_test, prob_test))
    
    
resultado2 = pd.DataFrame({"Target em Treino":target_train,"Target em Teste":target_test,"AUC em Treino":roc_train, "AUC em Teste":roc_test})
resultado2

Veja como os Targets ficaram. Agora a comparação entre os modelos ficou mais justa.

Compare os resultados:

In [ ]:
# K-Fold sem estratificação
resultado.describe().loc[['mean','std']][['AUC em Treino','AUC em Teste']]

In [ ]:
# K-Fold COM estratificação
resultado2.describe().loc[['mean','std']][['AUC em Treino','AUC em Teste']]

Veja como **a variação do AUC em Teste diminuiu bastante**. Isso mostra que a variação do AUC em Teste anterior estava sendo impactada pelas distribuições diferentes entre os Targets em Treino e Teste.

**Dica:** A estratificação é muito indicada nos casos em que temos uma base muito pequena ou com o target desbalanceado. Desta forma garantimos que ao menos teremos partições com um número próximo de classes 1 e 0.

**Para saber mais**

Existem outros tipos de validação cruzada menos frequentes na prática. Confira mais em: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection